In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from pprint import pprint as pp
import time


In [2]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import platform
path = None
if platform.system() is 'Windows':
    path = './drivers/geckodriver.exe'
else:
    path = './drivers/geckodriver'
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options,executable_path=path)

In [3]:
url_data = pd.read_csv("./dataset/url.csv")
url_data.head()

,url,date,group
0,https://www.taiwannews.com.tw/en/news/3819595,11/18/2019,0.0
1,https://twitter.com/Woppa1Woppa/status/1196315...,11/17/2019,0.0
2,https://www.washingtonpost.com/nation/2019/11/...,11/04/2019,1.0
3,https://www.sltrib.com/news/2019/11/01/utah-el...,11/01/2019,1.0
4,https://www.yahoo.com/news/principal-teacher-s...,11/05/2019,1.0


In [4]:
def html_decode(s):
    """
    Returns the ASCII decoded version of the given HTML string. This does
    NOT remove normal HTML tags like <p>.
    """
    htmlCodes = (
            ("'", '&#39;'),
            ('"', '&quot;'),
            ('>', '&gt;'),
            ('<', '&lt;'),
            ('&', '&amp;')
        )
    for code in htmlCodes:
        s = s.replace(code[1], code[0])
    return s

In [5]:
def get_page_source(url):
    driver.get(url)
    return driver.page_source

In [6]:
scrapping_url = "https://embed.ly/docs/explore/extract?url="
apikey = "1934f519a63e142e0d3c893e59cc37fe0172e98a"
sharedcount_url = "https://api.sharedcount.com/v1.0/?url={}&apikey={}"
reddit_search = "https://www.reddit.com/domain/{}/search?q={}&sort=top"
colon = '%3A'
fw_slash = '%2F'
def get_sharedcount_url(url):
    return sharedcount_url.format(url,apikey)

def get_fbengagement(url):
    shared_count = json.loads(requests.get(get_sharedcount_url(url)).text)
    if 'Facebook' not in shared_count.keys():
        return {}
    fb = shared_count['Facebook']
    return fb

def scrape_content(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    json_text = html_decode(soup.pre.text)
    row = json.loads(json_text)
    if row['content'] is None and row['description'] is None:
        print("Skipping link : ",url)
        return None
    if row['content'] is None and row['description'] is not None :
        row['content'] = row['description']
    row['content']= BeautifulSoup(row['content'], "lxml").text
    row['provider_display'] = row['provider_display'].replace('www.','')
    return row

def scrape_reddit(domain, title):
    query_url = reddit_search.format(domain,title.replace(" ","+"))
#     page = requests.get(query_url).text
    page = get_page_source(query_url)
    soup = BeautifulSoup(page,'html.parser')
    upvote = list()
    for score in soup.findAll('span',{'class':'search-score'}):
        upvote += [int(i) for i in score.text.split() if i.isdigit()] 
    comments = list()
    for score in soup.findAll('a',{'class':'search-comments'}):
        comments += [int(i) for i in score.text.split() if i.isdigit()] 
    return (sum(upvote),sum(comments))

def scrape_news(data):
    for url in data:
        url = url.replace(':',colon)
        url = url.replace('/',fw_slash)
        article = scrape_content(scrapping_url+url)
        if article is None:
            continue
        article.update(get_fbengagement(url))
        article['reddit_upvotes'],article['reddit_comments'] = scrape_reddit(article['provider_display'],article['title'])
        yield article

# for row in scrape_news(url_data['url']):
#     print(row['title'])
    


In [7]:
def create_data(data):
    dataset = list()
    i = 0
    for row in scrape_news(data):
        dataset+=[row]
    return dataset

In [8]:

df = pd.DataFrame(create_data(url_data['url']))
df = df[['language','title','content','description','provider_url','keywords','provider_display','provider_name',
            'total_count','comment_count','share_count','reaction_count','reddit_upvotes','reddit_comments']]
df['date'] = url_data['date']
df['date'] = pd.to_datetime(df['date'])
df['group'] = url_data['group']
df['title_len'] = df['title'].str.split().apply(len)
df['content_len'] = df['content'].str.split().apply(len)
df['description_len'] = df['description'].str.split().apply(len)

df.head()

Skipping link :  https://embed.ly/docs/explore/extract?url=https%3A%2F%2Fwww.theguardian.com%2Fus-news%2F2019%2Fnov%2F18%2Farkansas-chemistry-professors-meth-lab-breaking-bad


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
df.to_csv("./dataset/meta.csv")

In [ ]:
driver.quit()